In [1]:
#Download the content
import requests
#Parse 
from bs4 import BeautifulSoup
#Catch the content
from lxml import etree
#Extract content
import regex as re
#Lib spacy
import spacy
#Lib json
import json

In [2]:
RE_NORMALIZE_BLANKS = re.compile(r'[\s]+')
url = "http://www.planalto.gov.br/ccivil_03/constituicao/ConstituicaoCompilado.htm"
REQUEST_HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

#Clean the text
def replace_all_problems(text):
    fim = text.replace("\r\n", " ")
    return fim

#Normalize the text
def normalize_text(s):
    return RE_NORMALIZE_BLANKS.sub(' ', s).strip()

#Request the html
def download_html(url):
    r = requests.get(url, headers=REQUEST_HEADER)
    return r.text

#Split the html in the end of art 250
def remove_pos_normas_centrais(html):

    return html.split("Brasília, 5 de outubro de 1988.")[0]

#Parse the soup
def parse_soup(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup

#Extract the preambulo
def extract_preambulo(soup):
    dom = etree.HTML(str(soup))
    content = replace_all_problems((dom.xpath('/html/body/p[3]/font/small')[0].text))
    dictionary = {"preambulo": str(content)}
    return dictionary

def create_title(soup):
    find = soup.find_all("p", face="Arial")
    return find

html = download_html(url)
html_preambulo_e_normas_centrais = remove_pos_normas_centrais(html)
soup = parse_soup(html_preambulo_e_normas_centrais)

cf = ' '.join(p.text for p in soup.find_all('p'))
cf = normalize_text(cf)
cf = cf.strip()

In [90]:
FIND_TITLE = re.compile('TÍTULO\s+(?P<titulo_numero>[IXV]+).*?(?=TÍTULO|$)')
FIND_CHAPTER = re.compile('CAPÍTULO (?P<capitulo_numero>[IXV]+).*?(?=CAPÍTULO|$)')
FIND_ARTICLE = re.compile('Art\. (?P<artigo_numero>[123456789]+).*?(?=Art\.|CAPÍTULO|$)')

In [151]:
principal = {'titulos': []}

for match_titulo in FIND_TITLE.finditer(cf):
    
    titulo = {'titulo_identificador': match_titulo.group('titulo_numero'), 'capitulos': []}
    
    principal['titulos'].append(titulo)

    for match_capitulo in FIND_CHAPTER.finditer(str(match_titulo)):
        
        capitulo = {'capitulo_identificador': match_capitulo.group('capitulo_numero')}
        
        titulo['capitulos'].append(capitulo)
               
capitulo = { 'capitulo_identificador': '0'}
        
for titulo in principal['titulos']:
    if(len(titulo['capitulos']) < 1):
        titulo['capitulos'].append(capitulo)

principal

{'titulos': [{'titulo_identificador': 'I',
   'capitulos': [{'capitulo_identificador': '0'}]},
  {'titulo_identificador': 'II',
   'capitulos': [{'capitulo_identificador': 'I'},
    {'capitulo_identificador': 'II'},
    {'capitulo_identificador': 'III'},
    {'capitulo_identificador': 'IV'},
    {'capitulo_identificador': 'V'}]},
  {'titulo_identificador': 'III',
   'capitulos': [{'capitulo_identificador': 'I'},
    {'capitulo_identificador': 'II'},
    {'capitulo_identificador': 'III'},
    {'capitulo_identificador': 'IV'},
    {'capitulo_identificador': 'V'},
    {'capitulo_identificador': 'VI'},
    {'capitulo_identificador': 'VII'}]},
  {'titulo_identificador': 'IV',
   'capitulos': [{'capitulo_identificador': 'I'},
    {'capitulo_identificador': 'II'},
    {'capitulo_identificador': 'III'},
    {'capitulo_identificador': 'IV'}]},
  {'titulo_identificador': 'V',
   'capitulos': [{'capitulo_identificador': 'I'},
    {'capitulo_identificador': 'II'},
    {'capitulo_identificador': 'I